# Standarisation du process d'ingestions des fichiers de prévisions mensuelles pour les fichiers excel à multiple feuilles

In [1]:
%config Completer.use_jedi = False

In [2]:
import sys 
sys.path.append('/home/cdsw/code')
import imp
from utils import *
import numpy as np
import math
import openpyxl
import pandas as pd
import dateutil
from dateutil.parser import parse
import warnings
warnings.filterwarnings("ignore")
from Prev.multiple_sheet_process import * 

In [3]:
import Prev.multiple_sheet_process
import utils
imp.reload(utils)
imp.reload(Prev.multiple_sheet_process)
from utils import *
from Prev.multiple_sheet_process import * 

In [4]:
pd.set_option("max_rows", 5000)
pd.set_option('max_columns', 50)

In [5]:
emplacement = "/mnt/Plateforme/1 - Collecte des données sources auprès des OP/19 - Tresorerie/Prévisions DEA mensualisées"
extension = "*.xlsx"
list_filename = get_files_in_directory(emplacement,extension)
filename = list_filename[2]
print('le nom du fichier est ', filename)
annee = (openpyxl.load_workbook(filename).sheetnames)[0]
print("l'annee est ", annee )
df = pd.read_excel(filename, sheet_name = annee ,engine ='openpyxl')

le nom du fichier est  /mnt/Plateforme/1 - Collecte des données sources auprès des OP/19 - Tresorerie/Prévisions DEA mensualisées/Equilibre Technique mensualisé 2018.06 - DFT.xlsx
l'annee est  2018


## Travail sur les données

In [163]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,En millions d'euros,Janvier,Février,Mars,Avril,Mai,Juin,Juillet,Août,Septembre,Octobre,Novembre,Décembre,2011.0
1,RECETTES,Contributions et autres produits,3582.2,2278.7,2273.9,3093.7,2370.1,2315.8,3236.7,2431,2213.1,3134.7,2252.7,2605.8,31788.3
2,NaN,Contributions,3464.6,2247.1,2222,3059.4,2313.7,2262.7,3211.3,2404.1,2149.9,3082.2,2218.8,2557.3,31193.1
3,NaN,"Autres produits, conventions diverses",83.8,6.6,11.1,4.7,27.2,27.4,1.2,1.4,27.4,18.6,12.3,31.4,253.0
4,NaN,Participation entreprise CSP / CRP,33.8,24.9,40.9,29.6,29.2,25.6,24.1,25.6,35.9,33.9,21.7,17.1,342.2


In [164]:
if check_all_unnamed(df):
    new_header = df.iloc[0] 
    df = df[1:] #
    df.columns = new_header 

In [165]:
stop_l, stop_c = check_index(df)[0], check_index(df)[1]
df =df[df.columns[stop_c[0]:stop_c[1]]]
df =df[:stop_l]

In [166]:
df = df.T
df= df.reset_index(level=0)
df = df.rename(columns = {df.columns[0] : "mois_nom"})
df

,mois_nom,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
0,En millions d'euros,Contributions et autres produits,Contributions,"Autres produits, conventions diverses",Participation entreprise CSP / CRP,Gestion administrative,Revenus financiers,Total recettes,null,Dépenses allocataires,Allocations brutes,"ARE / AREF, y compris UE","CRP / CTP / CSP, y compris équivalents ARE","Indus, avances et acomptes","Autres allocations (décès, ...)",Aides,Aides à la reprise et création d'entreprise,"Autres aides (adr, aide fin de droit, ...)",Prélèvements sociaux,Caisses de retraites complémentaire,"Sécurité sociale (CSG, CRDS, maladie)",Reversements et compléments,Caisses de retraites complémentaire (dont préc...,"Sécurité sociale (CSG, CRDS, maladie)",Dépenses autres publics,Allocation partielle de longue durée (APLD),"Autres (AS-FNE, Plan rebond, ...)",Financement et frais de gestion opérateurs,10% Pôle Emploi,"Autres (accompagnement CSP, recouvrements)",Frais de gestion et décaissements sur immobili...,Intérêts nets sur emprunts,Total Dépenses
1,Janvier,3582.2,3464.6,83.8,33.8,2,2,3584.2,NaN,3185.7,2417.7,2360.2,129.7,-72.7,0.4,94.2,84,10.3,-149.1,-98.5,-50.7,822.9,772.3,50.6,5.8,0,5.8,256.5,247.1,9.4,3.4,3.8,3455.2
2,Février,2278.7,2247.1,6.6,24.9,4.4,0.1,2283.1,NaN,2562.2,2454.1,2399.9,126.7,-73.1,0.5,85.1,74.7,10.5,-150.3,-100.1,-50.3,173.3,123.1,50.2,0.6,0,0.6,257,247.1,9.8,4,3.6,2827.4
3,Mars,2273.9,2222,11.1,40.9,0.4,0.2,2274.4,NaN,2258.8,2214.5,2171.9,112.8,-70.7,0.5,106.2,94.6,11.7,-137,-89.9,-47,75.1,28.1,47,3.2,0,3.2,251,247.1,3.9,3.3,6.8,2523.2
4,Avril,3093.7,3059.4,4.7,29.6,0.6,0.8,3094.2,NaN,2916.9,2319.7,2264,118,-62.7,0.3,92.5,82.5,10,-143.9,-94.8,-49.1,648.6,599.5,49,1.5,0,1.5,251,247.1,3.9,6.7,15.5,3191.7
5,Mai,2370.1,2313.7,27.2,29.2,0.3,0.1,2370.4,NaN,2231.3,2195.9,2150.5,112.2,-67.3,0.4,99.7,89.1,10.6,-136.9,-89.4,-47.5,72.6,25.1,47.5,0.9,0,0.9,256.1,247.1,8.9,7.4,8.1,2503.7
6,Juin,2315.8,2262.7,27.4,25.6,8.1,0,2323.9,NaN,2219.3,2197.3,2147.9,112.4,-63.3,0.3,85.6,75.6,9.9,-137,-89.9,-47.1,73.4,26.4,47,1.4,0,1.4,255.1,247.1,7.9,2.8,5.2,2483.7
7,Juillet,3236.7,3211.3,1.2,24.1,16.9,0.7,3253.6,NaN,2547.5,2094.4,2050.2,107.3,-63.4,0.3,93.8,84.3,9.6,-131,-85.8,-45.1,490.3,445.2,45.1,0.9,0,0.9,252.1,247.1,4.9,3.4,5.8,2809.7
8,Août,2431,2404.1,1.4,25.6,0.1,1,2431.1,NaN,2236.3,2215.5,2172.9,111.6,-69.4,0.4,88.9,78.5,10.4,-136.3,-90.5,-45.8,68.2,22.4,45.8,0.8,0,0.8,249.1,247.1,2,2.9,4.3,2493.5
9,Septembre,2213.1,2149.9,27.4,35.9,8.4,0.2,2221.6,NaN,2376.5,2366.8,2323.5,115.8,-72.9,0.4,85.7,75.4,10.3,-144.4,-97.3,-47.2,68.4,21.3,47.1,1.1,0,1.1,250.1,247.1,3,5.3,4.7,2637.6


In [167]:
# remplir la colonne année 

df['annee'] = annee



In [168]:
df[['annee','mois_nom', df.columns[1]] + df.columns[2:-1].tolist()]


,annee,mois_nom,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
0,2011,En millions d'euros,Contributions et autres produits,Contributions,"Autres produits, conventions diverses",Participation entreprise CSP / CRP,Gestion administrative,Revenus financiers,Total recettes,null,Dépenses allocataires,Allocations brutes,"ARE / AREF, y compris UE","CRP / CTP / CSP, y compris équivalents ARE","Indus, avances et acomptes","Autres allocations (décès, ...)",Aides,Aides à la reprise et création d'entreprise,"Autres aides (adr, aide fin de droit, ...)",Prélèvements sociaux,Caisses de retraites complémentaire,"Sécurité sociale (CSG, CRDS, maladie)",Reversements et compléments,Caisses de retraites complémentaire (dont préc...,"Sécurité sociale (CSG, CRDS, maladie)",Dépenses autres publics,Allocation partielle de longue durée (APLD),"Autres (AS-FNE, Plan rebond, ...)",Financement et frais de gestion opérateurs,10% Pôle Emploi,"Autres (accompagnement CSP, recouvrements)",Frais de gestion et décaissements sur immobili...,Intérêts nets sur emprunts,Total Dépenses
1,2011,Janvier,3582.2,3464.6,83.8,33.8,2,2,3584.2,NaN,3185.7,2417.7,2360.2,129.7,-72.7,0.4,94.2,84,10.3,-149.1,-98.5,-50.7,822.9,772.3,50.6,5.8,0,5.8,256.5,247.1,9.4,3.4,3.8,3455.2
2,2011,Février,2278.7,2247.1,6.6,24.9,4.4,0.1,2283.1,NaN,2562.2,2454.1,2399.9,126.7,-73.1,0.5,85.1,74.7,10.5,-150.3,-100.1,-50.3,173.3,123.1,50.2,0.6,0,0.6,257,247.1,9.8,4,3.6,2827.4
3,2011,Mars,2273.9,2222,11.1,40.9,0.4,0.2,2274.4,NaN,2258.8,2214.5,2171.9,112.8,-70.7,0.5,106.2,94.6,11.7,-137,-89.9,-47,75.1,28.1,47,3.2,0,3.2,251,247.1,3.9,3.3,6.8,2523.2
4,2011,Avril,3093.7,3059.4,4.7,29.6,0.6,0.8,3094.2,NaN,2916.9,2319.7,2264,118,-62.7,0.3,92.5,82.5,10,-143.9,-94.8,-49.1,648.6,599.5,49,1.5,0,1.5,251,247.1,3.9,6.7,15.5,3191.7
5,2011,Mai,2370.1,2313.7,27.2,29.2,0.3,0.1,2370.4,NaN,2231.3,2195.9,2150.5,112.2,-67.3,0.4,99.7,89.1,10.6,-136.9,-89.4,-47.5,72.6,25.1,47.5,0.9,0,0.9,256.1,247.1,8.9,7.4,8.1,2503.7
6,2011,Juin,2315.8,2262.7,27.4,25.6,8.1,0,2323.9,NaN,2219.3,2197.3,2147.9,112.4,-63.3,0.3,85.6,75.6,9.9,-137,-89.9,-47.1,73.4,26.4,47,1.4,0,1.4,255.1,247.1,7.9,2.8,5.2,2483.7
7,2011,Juillet,3236.7,3211.3,1.2,24.1,16.9,0.7,3253.6,NaN,2547.5,2094.4,2050.2,107.3,-63.4,0.3,93.8,84.3,9.6,-131,-85.8,-45.1,490.3,445.2,45.1,0.9,0,0.9,252.1,247.1,4.9,3.4,5.8,2809.7
8,2011,Août,2431,2404.1,1.4,25.6,0.1,1,2431.1,NaN,2236.3,2215.5,2172.9,111.6,-69.4,0.4,88.9,78.5,10.4,-136.3,-90.5,-45.8,68.2,22.4,45.8,0.8,0,0.8,249.1,247.1,2,2.9,4.3,2493.5
9,2011,Septembre,2213.1,2149.9,27.4,35.9,8.4,0.2,2221.6,NaN,2376.5,2366.8,2323.5,115.8,-72.9,0.4,85.7,75.4,10.3,-144.4,-97.3,-47.2,68.4,21.3,47.1,1.1,0,1.1,250.1,247.1,3,5.3,4.7,2637.6


In [11]:
df1= df[df.columns[:3]]

# changer l'entete de la table

df1= df1[1:].reset_index().drop(columns = 'index')
df1 = df1.rename(columns = {0:'type_montant'})

df1['key'] = [i for i in range(len(df1))]

df1

,annee,mois_nom,type_montant,key
0,2011,Unnamed: 2,Janvier,0
1,2011,Unnamed: 3,Février,1
2,2011,Unnamed: 4,Mars,2
3,2011,Unnamed: 5,Avril,3
4,2011,Unnamed: 6,Mai,4
5,2011,Unnamed: 7,Juin,5
6,2011,Unnamed: 8,Juillet,6
7,2011,Unnamed: 9,Août,7
8,2011,Unnamed: 10,Septembre,8
9,2011,Unnamed: 11,Octobre,9


In [12]:
df2= df[df.columns[3:]]

new_header = df2.iloc[0] #grab the first row for the header
df2 = df2[1:] #take the data less the header row
df2.columns = new_header #set the header row as the df header

new_columns = [c for c in df2.columns if type(c) == str]
df2 =df2[new_columns]

In [13]:
df2

,Contributions et autres produits,Contributions,"Autres produits, conventions diverses",Participation entreprise CSP / CRP,Gestion administrative,Revenus financiers,Total recettes,Dépenses allocataires,Allocations brutes,"ARE / AREF, y compris UE",...,"Sécurité sociale (CSG, CRDS, maladie)",Dépenses autres publics,Allocation partielle de longue durée (APLD),"Autres (AS-FNE, Plan rebond, ...)",Financement et frais de gestion opérateurs,10% Pôle Emploi,"Autres (accompagnement CSP, recouvrements)",Frais de gestion et décaissements sur immobilisations,Intérêts nets sur emprunts,Total Dépenses
1,3582.2,3464.6,83.8,33.8,2,2,3584.2,3185.7,2417.7,2360.2,...,50.6,5.8,0,5.8,256.5,247.1,9.4,3.4,3.8,3455.2
2,2278.7,2247.1,6.6,24.9,4.4,0.1,2283.1,2562.2,2454.1,2399.9,...,50.2,0.6,0,0.6,257,247.1,9.8,4,3.6,2827.4
3,2273.9,2222,11.1,40.9,0.4,0.2,2274.4,2258.8,2214.5,2171.9,...,47,3.2,0,3.2,251,247.1,3.9,3.3,6.8,2523.2
4,3093.7,3059.4,4.7,29.6,0.6,0.8,3094.2,2916.9,2319.7,2264,...,49,1.5,0,1.5,251,247.1,3.9,6.7,15.5,3191.7
5,2370.1,2313.7,27.2,29.2,0.3,0.1,2370.4,2231.3,2195.9,2150.5,...,47.5,0.9,0,0.9,256.1,247.1,8.9,7.4,8.1,2503.7
6,2315.8,2262.7,27.4,25.6,8.1,0,2323.9,2219.3,2197.3,2147.9,...,47,1.4,0,1.4,255.1,247.1,7.9,2.8,5.2,2483.7
7,3236.7,3211.3,1.2,24.1,16.9,0.7,3253.6,2547.5,2094.4,2050.2,...,45.1,0.9,0,0.9,252.1,247.1,4.9,3.4,5.8,2809.7
8,2431,2404.1,1.4,25.6,0.1,1,2431.1,2236.3,2215.5,2172.9,...,45.8,0.8,0,0.8,249.1,247.1,2,2.9,4.3,2493.5
9,2213.1,2149.9,27.4,35.9,8.4,0.2,2221.6,2376.5,2366.8,2323.5,...,47.1,1.1,0,1.1,250.1,247.1,3,5.3,4.7,2637.6
10,3134.7,3082.2,18.6,33.9,6.8,3.1,3141.5,2803.9,2216,2175.3,...,45.5,21,20.4,0.6,252.4,247.1,5.2,6.1,0.6,3084


In [14]:

df3= df2.T.unstack().reset_index(level=1, name='montant_millions')
df3= df3.rename(columns ={0:'nature_montant'})

l = [len(df2.columns)*[i] for i in range(len(df1))]
flat_l = [item for sublist in l for item in sublist]

df3['key'] = flat_l



In [15]:
dff = df3.merge(df1, on= 'key')
dff.drop(columns ='key', inplace = True)
dff['montant_millions'] = dff['montant_millions'].astype(float, errors = 'ignore')


In [16]:
mapping_month = {'janvier': 1, 'février':2 , 'mars': 3, 'avril': 4, 'mai':5, 'juin':6 , 'juillet':7 , 'août' : 8 ,
                 'septembre' :9 , 'octobre' : 10 , 'novembre' : 11 , 'décembre' : 12 }

dff['mois'] = dff['mois_nom'].map(mapping_month)
dff['mois'] = dff['mois'].astype(int, errors ='ignore')

In [17]:
dff

,nature_montant,montant_millions,annee,mois_nom,type_montant,mois
0,Contributions et autres produits,3582.2,2011,Unnamed: 2,Janvier,NaN
1,Contributions,3464.6,2011,Unnamed: 2,Janvier,NaN
2,"Autres produits, conventions diverses",83.8,2011,Unnamed: 2,Janvier,NaN
3,Participation entreprise CSP / CRP,33.8,2011,Unnamed: 2,Janvier,NaN
4,Gestion administrative,2.0,2011,Unnamed: 2,Janvier,NaN
5,Revenus financiers,2.0,2011,Unnamed: 2,Janvier,NaN
6,Total recettes,3584.2,2011,Unnamed: 2,Janvier,NaN
7,Dépenses allocataires,3185.7,2011,Unnamed: 2,Janvier,NaN
8,Allocations brutes,2417.7,2011,Unnamed: 2,Janvier,NaN
9,"ARE / AREF, y compris UE",2360.2,2011,Unnamed: 2,Janvier,NaN


In [18]:
version_prev = get_date_extraction_m_yy(filename)
dff['version_prev'] = version_prev
dff['version_prev'] = pd.to_datetime(dff['version_prev'])
dff['version_prev']= dff['version_prev'].dt.strftime('%Y-%m')

In [19]:
dff = dff[['version_prev','annee' , 'mois' , 'mois_nom', 'nature_montant',  'type_montant', 'montant_millions' ]]
dff

,version_prev,annee,mois,mois_nom,nature_montant,type_montant,montant_millions
0,2018-02,2011,NaN,Unnamed: 2,Contributions et autres produits,Janvier,3582.2
1,2018-02,2011,NaN,Unnamed: 2,Contributions,Janvier,3464.6
2,2018-02,2011,NaN,Unnamed: 2,"Autres produits, conventions diverses",Janvier,83.8
3,2018-02,2011,NaN,Unnamed: 2,Participation entreprise CSP / CRP,Janvier,33.8
4,2018-02,2011,NaN,Unnamed: 2,Gestion administrative,Janvier,2.0
5,2018-02,2011,NaN,Unnamed: 2,Revenus financiers,Janvier,2.0
6,2018-02,2011,NaN,Unnamed: 2,Total recettes,Janvier,3584.2
7,2018-02,2011,NaN,Unnamed: 2,Dépenses allocataires,Janvier,3185.7
8,2018-02,2011,NaN,Unnamed: 2,Allocations brutes,Janvier,2417.7
9,2018-02,2011,NaN,Unnamed: 2,"ARE / AREF, y compris UE",Janvier,2360.2


# Code sous forme de fonction

In [6]:
normal_values_nature_montant = ['Autres dépenses allocations','Impositions de toutes natures', '     dont prélèvement PAS',
 'Pôle emploi - Allocataires','     dont reversement PAS','Pour dépenses Unédic (part tiers)', 'ASP - Activité partielle',
 'Dépenses autres publics','ARE / AREF','    Aides','Cumul inversé (à fin de mois)','Caisses de retraite complémentaire',
 'Total dépenses', 'Participation entreprise CSP / CRP','Somme retraitée','Raisons','Pôle emploi - CSP', 'Autres produits',
 'Acoss - CEA (chèque emploi-associatif)', '    Allocations brutes','Pôle emploi - PAS allocataires',
 'Allocation partielle de longue durée (APLD)', 'Autres (accompagnement CSP, recouvrements)', 'Somme autres',
 'Total dépenses - Equilibre technique','Solde = recettes - dépeneses', 'Variation de tréso','Caisse de retraites net',
 'Contributions','  dont surplus de dépenses crise* ','Acoss - TREC (part patronale)','dont intérêt', 'Dépenses ARE AREF ASP ',
 'Éléments exceptionnels', '    Prélèvements sociaux','Autres aides (adr, aide fin de droit, ...)', 'ATI','11% Pôle Emploi',
 'Recettes yc crise','Pôle emploi - Participation Unédic (11%)', 'Ajustement', 'Gestion administrative',
 'Variation de trésorerie',"Remboursements d'indus",'Activité partielle','Somme','dont Pole emploi', 'AREP',
 'Autres (AS-FNE, Plan rebond, ...)','Indus, avances et acomptes','Recettes techniques','Cumul','     dont reversement secu',
 'Contributions et autres produits','Calculé', "En millions d'euros",'Autres allocations (décès, ...)',
 "       dont manque à gagner Activité partielle + garde d'enfant","Contributions d'Assurance chômage",'Total',
 'Autres', '    Prélèvements sociaux et fiscaux','Somme brute','Variation de trésorerie - Equilibre technique',
 'Caisses de retraite complémentaire (dont précomptes)','Dépenses yc crise','    Reversements et compléments',
 'Intérêts', 'Retraitements','Acoss - EPM (particuliers employeurs)','Sécurité sociale (CSG, CRDS, maladie)',
 'CRP / CTP / CSP, y compris équivalents ARE','Total recettes','ARE / AREF, y compris UE','Somme dépenses techniques',
 'CRP / CSP, y compris équivalents ARE','  dont prélèvement','Frais de gestion et décaissements sur immobilisations',
 'Financement et frais de gestion opérateurs','Autres produits (gestion administrative et financière)',
 'Pôle emploi - URSAFF allocataires','Pour dépenses Unédic (dépenses allocataires)',
 '10% Pôle Emploi','Intérêts nets sur emprunts','Frais de fonctionnement',
 'Autres dépenses','Autres aides  (prime CSP, aide fin de droit, ...)',
 "Contributions et autres recettes d'Assurance chômage",'Revenus financiers', 'dont frais de fonctionnement et de gestion',
 'Prélèvement à la source','Caisses de retraites complémentaire (dont précomptes)',
 '    dont prélèvement sécu', 'Dépenses allocataires','Pôle emploi - Contribution (11%)', 'Total Dépenses',
 'Autres aides (prime CSP, aide fin de droit, ...)','Equilibre technique','Frais de fonctionnement et de gestion',
 'Autres produits, conventions diverses','Dépenses techniques',
 'Eléments exceptionnels','Etat - Prélèvement à la source',"Aides à la reprise et création d'entreprise",'Contributions principales',
 'Activité partielle, allocation partielle de longue durée (APLD)','Intérêts sur emprunts',
 'Conventions diverses, y compris UE', 'Impositions de toutes natures (CSG)',
 'Caisses de retraites complémentaire', '  dont Reversement hors effet crise', 'Pour recettes Unédic','Somme cumulée',
 'Caisses de retraite allocataires', 'Raison','       dont décalage de trésorerie',
 'Total recettes - Equilibre technique','Acoss - Réduction générale','Acoss - CSG activité' ]

In [9]:
emplacement = "/mnt/Plateforme/1 - Collecte des données sources auprès des OP/19 - Tresorerie/Prévisions DEA mensualisées"
extension = "*.xlsx"
list_filename = get_files_in_directory(emplacement,extension)

filename = list_filename[6]
filename


'/mnt/Plateforme/1 - Collecte des données sources auprès des OP/19 - Tresorerie/Prévisions DEA mensualisées/Equilibre Technique mensualisé 2019 11 - DFT.xlsx'

In [10]:
dataframe = prevision_data_ingestion_multiple_sheet(filename, '.xlsx',normal_values_nature_montant)
dataframe

TypeError: prevision_data_ingestion_multiple_sheet() missing 5 required positional arguments: 'table_name', 'filename', 'extension', 'normal_values_nature_montant', and 'log'